# Dimensional reduction

## Ca原子座標の主成分分析


In [ ]:
import mdtraj as md
import numpy as np

w_ref = md.load('../md/protein.pdb')
w = md.load('../md/protein.dcd', top='../md/protein.pdb')

In [ ]:
atoms_to_keep = [a.index for a in w.topology.atoms if a.name == 'CA']
w.restrict_atoms(atoms_to_keep)
w_ref.restrict_atoms(atoms_to_keep)

In [3]:
traj = w

In [ ]:
traj.xyz.shape

In [ ]:
import copy
ref = copy.copy(traj[0])
for i in range(10):
    traj.superpose(ref)
    mean_xyz = np.mean(traj.xyz, axis=0, keepdims=True)
    print(np.square(np.sum((ref.xyz - mean_xyz)**2)))
    ref.xyz = mean_xyz

In [ ]:
traj.superpose(ref, 0)

In [ ]:
traj.xyz.shape

In [ ]:
coordinates = traj.xyz.reshape(traj.n_frames, traj.n_atoms*3)
coordinates.shape

In [9]:
from sklearn.decomposition import PCA
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt

pca = PCA()
pca_result = pca.fit_transform(coordinates)

In [ ]:
# Plot the first two principal components
plt.figure(figsize=(10, 7))
plt.scatter(x=pca_result[:, 0], marker='.', y=pca_result[:, 1], label="wildtype")
plt.legend()
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA scatter plot')

In [ ]:
# Calculate the potential of mean force
kde = gaussian_kde(np.vstack([pca_result[:, 0], pca_result[:, 1]]))

# Compute the potential of mean force
x, y = np.mgrid[min(pca_result[:, 0]):max(pca_result[:, 0]):100j,
                min(pca_result[:, 1]):max(pca_result[:, 1]):100j]
positions = np.vstack([x.ravel(), y.ravel()])
pmf = np.reshape(kde(positions), x.shape)

# Plot the potential of mean force as contours on top of the scatter plot
plt.figure(figsize=(10, 7))
pmf_normalized = -np.log(pmf) - np.min(-np.log(pmf))
plt.contourf(x, y, pmf_normalized, np.arange(0.0, 8.0, 0.5), cmap='viridis')
plt.colorbar(label='Free Energy (kT)')
plt.title('Free energy sruface (Wild-type)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()
